In [ ]:
import math
import numpy as np
import pyrosim

HEIGHT = 0.3
RADIUS = 0.05

seconds = 10.0
dt = 0.05
eval_time = int(seconds/dt)
gravity = -1.0

sim = pyrosim.Simulator(eval_time=eval_time, dt=dt, gravity=gravity,
                        debug=True, play_paused=True,
                        play_blind=False, use_textures=True)

In [ ]:
main_body = sim.send_box(x=0, y=0, z=HEIGHT+RADIUS,
                         length=HEIGHT, width=HEIGHT,
                         height=RADIUS*2.0, mass=1)
light_sensor = sim.send_light_sensor(body_id=main_body)

# id arrays
thighs = [0]*4
shins = [0]*4
hips = [0]*4
knees = [0]*4
foot_sensors = [0]*4
sensor_neurons = [0]*5
motor_neurons = [0]*8

delta = float(math.pi)/2.0

# quadruped is a box with one leg on each side
# each leg consists thigh and shin cylinders
# with hip and knee joints
# each shin/foot then has a touch sensor
for i in range(4):
    theta = delta*i
    x_pos = math.cos(theta)*HEIGHT
    y_pos = math.sin(theta)*HEIGHT

    thighs[i] = sim.send_cylinder(x=x_pos, y=y_pos, z=HEIGHT+RADIUS,
                                  r1=x_pos, r2=y_pos, r3=0,
                                  length=HEIGHT, radius=RADIUS, capped=True
                                  )

    hips[i] = sim.send_hinge_joint(main_body, thighs[i],
                                   x=x_pos/2.0, y=y_pos/2.0, z=HEIGHT+RADIUS,
                                   n1=-y_pos, n2=x_pos, n3=0,
                                   lo=-math.pi/4.0, hi=math.pi/4.0,
                                   speed=1.0)

    x_pos2 = math.cos(theta)*1.5*HEIGHT
    y_pos2 = math.sin(theta)*1.5*HEIGHT

    shins[i] = sim.send_cylinder(x=x_pos2, y=y_pos2, z=(HEIGHT+RADIUS)/2.0,
                                 r1=0, r2=0, r3=1,
                                 length=HEIGHT, radius=RADIUS,
                                 mass=1., capped=True)

    knees[i] = sim.send_hinge_joint(thighs[i], shins[i],
                                    x=x_pos2, y=y_pos2, z=HEIGHT+RADIUS,
                                    n1=-y_pos, n2=x_pos, n3=0,
                                    lo=-math.pi/4.0, hi=math.pi/4.0)

    foot_sensors[i] = sim.send_touch_sensor(shins[i])
    sensor_neurons[i] = sim.send_sensor_neuron(foot_sensors[i])
    
    motor_neurons[i] = sim.send_motor_neuron(hips[i])
    motor_neurons[i+4] = sim.send_motor_neuron(knees[i])


sensor_neurons[-1] = sim.send_sensor_neuron(light_sensor)
for source_id in sensor_neurons:
    for target_id in motor_neurons:
        sim.send_synapse(source_id, target_id, np.random.randn())
        

env_box = sim.send_box(x=0, y=HEIGHT*30, z=HEIGHT/2.0,
                           length=HEIGHT, width=HEIGHT, height=HEIGHT)
sim.send_light_source(env_box)

In [ ]:
sim.start()
sim.wait_to_finish()

In [ ]:
pos_x_data = sim.get_sensor_data(pos_sensor, svi=0)
pos_y_data = sim.get_sensor_data(pos_sensor, svi=1)
pos_z_data = sim.get_sensor_data(pos_sensor, svi=2)
touch_data = sim.get_sensor_data(foot_sensors[0])

In [ ]:
for (x, y, z) in zip(pos_x_data, pos_y_data, pos_z_data):
    print(x, ", ", y, ", ", z)

In [ ]:
print(touch_data)